# Computational Hydrogen Electrode Tutorial

Import useful modules

In [23]:
import numpy as np
import matplotlib.pyplot as plt
from ase.io import read
from ase import Atoms
from ase.build import mx2, molecule
from ase.constraints import FixAtoms
from ase.visualize import view
from ase.calculators.espresso import Espresso, EspressoProfile
import time

In [42]:
MoS2 = mx2(formula='MoS2', kind='2H', a=3.18, thickness=3.19, size=(2, 2, 1), vacuum=10.)

In [43]:
MoS2.positions

array([[ 0.        ,  0.        , 11.595     ],
       [ 1.59      ,  0.91798693, 13.19      ],
       [ 1.59      ,  0.91798693, 10.        ],
       [-1.59      ,  2.75396078, 11.595     ],
       [ 0.        ,  3.67194771, 13.19      ],
       [ 0.        ,  3.67194771, 10.        ],
       [ 3.18      ,  0.        , 11.595     ],
       [ 4.77      ,  0.91798693, 13.19      ],
       [ 4.77      ,  0.91798693, 10.        ],
       [ 1.59      ,  2.75396078, 11.595     ],
       [ 3.18      ,  3.67194771, 13.19      ],
       [ 3.18      ,  3.67194771, 10.        ]])

In [44]:
# Optionally create profile to override paths in ASE configuration:
profile = EspressoProfile(
    command='mpirun -np 4 /Users/oliviero/PWSCF/espresso-git/bin/pw.x', pseudo_dir='./pseudos/'
)

In [45]:
pseudopotentials = {
    "H":"H.pbe-rrkjus_psl.1.0.0.UPF",
    "O":"O.pbe-n-kjpaw_psl.0.1.UPF",
    "Mo":"Mo_ONCV_PBE-1.0.oncvpsp.upf",
    "S":"s_pbe_v1.4.uspp.F.UPF"
}

In [122]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2.calc = calc

In [123]:
eMoS2 = MoS2.get_potential_energy()

In [124]:
print(eMoS2)

-10094.420618981774


In [125]:
MoS2 = read('MoS2_vcrelax.out')

In [126]:
MoS2.center()

In [127]:
MoS2.positions

array([[ 8.00261267e-04,  9.14688188e-01,  6.35012543e+00],
       [ 1.58511123e+00,  1.82998009e+00,  7.94349870e+00],
       [ 1.58511128e+00,  1.82997906e+00,  4.75675372e+00],
       [-1.58573460e+00,  3.66043743e+00,  6.35012558e+00],
       [-2.00093063e-03,  4.57641766e+00,  7.94411429e+00],
       [-2.00114573e-03,  4.57641758e+00,  4.75613867e+00],
       [ 3.17064002e+00,  9.15897827e-01,  6.35012497e+00],
       [ 4.75648140e+00,  1.82987738e+00,  7.94453638e+00],
       [ 4.75648244e+00,  1.82987791e+00,  4.75571733e+00],
       [ 1.58410661e+00,  3.66164705e+00,  6.35012469e+00],
       [ 3.16937385e+00,  4.57630852e+00,  7.94515188e+00],
       [ 3.16937409e+00,  4.57630853e+00,  4.75510112e+00]])

In [133]:
atop_index = 1

In [112]:
MoS2.positions[atop_index]

array([1.58511123, 1.82998009, 7.9434987 ])

In [140]:
fixed = list(range(len(MoS2)))
fixed.remove(atop_index)
print(fixed)
constraint = FixAtoms(indices=fixed)
MoS2.set_constraint(constraint)

[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


In [134]:
# Creating the OH molecule
oh_molecule = Atoms('OH', positions=[(0, 0, 0), (0, -0.763, 0.596)])

oh_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.87))

oh_molecule.positions

array([[ 1.58511123,  1.82998009,  9.4434987 ],
       [ 1.58511123,  1.06698009, 10.0394987 ]])

In [135]:
MoS2OH = MoS2 + oh_molecule
MoS2OH.positions

array([[ 8.00261267e-04,  9.14688188e-01,  6.35012543e+00],
       [ 1.58511123e+00,  1.82998009e+00,  7.94349870e+00],
       [ 1.58511128e+00,  1.82997906e+00,  4.75675372e+00],
       [-1.58573460e+00,  3.66043743e+00,  6.35012558e+00],
       [-2.00093063e-03,  4.57641766e+00,  7.94411429e+00],
       [-2.00114573e-03,  4.57641758e+00,  4.75613867e+00],
       [ 3.17064002e+00,  9.15897827e-01,  6.35012497e+00],
       [ 4.75648140e+00,  1.82987738e+00,  7.94453638e+00],
       [ 4.75648244e+00,  1.82987791e+00,  4.75571733e+00],
       [ 1.58410661e+00,  3.66164705e+00,  6.35012469e+00],
       [ 3.16937385e+00,  4.57630852e+00,  7.94515188e+00],
       [ 3.16937409e+00,  4.57630853e+00,  4.75510112e+00],
       [ 1.58511123e+00,  1.82998009e+00,  9.44349870e+00],
       [ 1.58511123e+00,  1.06698009e+00,  1.00394987e+01]])

In [136]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2OH'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2OH.calc = calc

In [120]:
from ase.optimize import QuasiNewton

In [137]:
dyn = QuasiNewton(MoS2OH, trajectory='MoS2OH.traj')
t = time.time()
dyn.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 19:07:08   -10674.144254       6.0320
BFGSLineSearch:    1[  2] 19:11:54   -10674.613325       2.0485
BFGSLineSearch:    2[  4] 19:16:09   -10674.660677       0.8835
BFGSLineSearch:    3[  5] 20:16:42   -10674.740613       0.9519
BFGSLineSearch:    4[  7] 20:21:27   -10674.784438       0.6007
BFGSLineSearch:    5[  9] 20:26:14   -10674.819318       1.4818
BFGSLineSearch:    6[ 10] 20:28:16   -10674.850648       0.4904
BFGSLineSearch:    7[ 11] 20:30:59   -10674.869527       0.2305
BFGSLineSearch:    8[ 12] 20:33:11   -10674.874348       0.1777
BFGSLineSearch:    9[ 13] 20:35:31   -10674.877357       0.1420
BFGSLineSearch:   10[ 15] 20:39:27   -10674.880650       0.0997
BFGSLineSearch:   11[ 18] 20:45:42   -10674.898461       0.7778
BFGSLineSearch:   12[ 19] 20:47:37   -10674.926511       0.9427
BFGSLineSearch:   13[ 20] 20:49:27   -10674.950072       1.4237
BFGSLineSearch:   14[ 21] 20:51:21   -1

In [141]:
print("bond S-O: ", MoS2OH.get_distance(1,12))
print("bond O-H: ", MoS2OH.get_distance(12,13))

bond S-O:  1.8715196542582317
bond O-H:  0.9800302812359026


In [142]:
# Creating the OH molecule
o_molecule = Atoms('O', positions=[(0, 0, 0)])

o_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.872))

o_molecule.positions

array([[1.58511123, 1.82998009, 9.7934987 ]])

In [143]:
MoS2O = MoS2 + o_molecule
MoS2O.positions

array([[ 8.00261267e-04,  9.14688188e-01,  6.35012543e+00],
       [ 1.58511123e+00,  1.82998009e+00,  7.94349870e+00],
       [ 1.58511128e+00,  1.82997906e+00,  4.75675372e+00],
       [-1.58573460e+00,  3.66043743e+00,  6.35012558e+00],
       [-2.00093063e-03,  4.57641766e+00,  7.94411429e+00],
       [-2.00114573e-03,  4.57641758e+00,  4.75613867e+00],
       [ 3.17064002e+00,  9.15897827e-01,  6.35012497e+00],
       [ 4.75648140e+00,  1.82987738e+00,  7.94453638e+00],
       [ 4.75648244e+00,  1.82987791e+00,  4.75571733e+00],
       [ 1.58410661e+00,  3.66164705e+00,  6.35012469e+00],
       [ 3.16937385e+00,  4.57630852e+00,  7.94515188e+00],
       [ 3.16937409e+00,  4.57630853e+00,  4.75510112e+00],
       [ 1.58511123e+00,  1.82998009e+00,  9.79349870e+00]])

In [144]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2O'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2O.calc = calc

In [145]:
dyn = QuasiNewton(MoS2O, trajectory='MoS2O.traj')
t = time.time()
dyn.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 21:05:51   -10658.228163       5.8142
BFGSLineSearch:    1[  1] 21:07:37   -10659.521952       2.3868
BFGSLineSearch:    2[  2] 21:09:34   -10659.764504       1.3492
BFGSLineSearch:    3[  4] 21:13:31   -10659.843787       0.4945
BFGSLineSearch:    4[  6] 21:17:58   -10659.846823       0.0452
Calculation time: 14.035029915968577 min.


In [146]:
print("bond S-O: ", MoS2OH.get_distance(1,12))

bond S-O:  1.8715196542582317


In [183]:
ooh_molecule = Atoms('OOH', positions=[(0, 0, 0), (0.75, 0.34, 1.10), (0.1, 0.25, 1.85)])

ooh_molecule.translate(MoS2.positions[atop_index] + (0, 0, 1.87))

ooh_molecule.positions

array([[ 1.58511123,  1.82998009,  9.8134987 ],
       [ 2.33511123,  2.16998009, 10.9134987 ],
       [ 1.68511123,  2.07998009, 11.6634987 ]])

In [184]:
MoS2OOH = MoS2 + ooh_molecule
MoS2OOH.positions

array([[ 8.00261267e-04,  9.14688188e-01,  6.35012543e+00],
       [ 1.58511123e+00,  1.82998009e+00,  7.94349870e+00],
       [ 1.58511128e+00,  1.82997906e+00,  4.75675372e+00],
       [-1.58573460e+00,  3.66043743e+00,  6.35012558e+00],
       [-2.00093063e-03,  4.57641766e+00,  7.94411429e+00],
       [-2.00114573e-03,  4.57641758e+00,  4.75613867e+00],
       [ 3.17064002e+00,  9.15897827e-01,  6.35012497e+00],
       [ 4.75648140e+00,  1.82987738e+00,  7.94453638e+00],
       [ 4.75648244e+00,  1.82987791e+00,  4.75571733e+00],
       [ 1.58410661e+00,  3.66164705e+00,  6.35012469e+00],
       [ 3.16937385e+00,  4.57630852e+00,  7.94515188e+00],
       [ 3.16937409e+00,  4.57630853e+00,  4.75510112e+00],
       [ 1.58511123e+00,  1.82998009e+00,  9.81349870e+00],
       [ 2.33511123e+00,  2.16998009e+00,  1.09134987e+01],
       [ 1.68511123e+00,  2.07998009e+00,  1.16634987e+01]])

In [185]:
input_data = {
    'control': {
        'restart_mode': 'from_scratch',
        'pseudo_dir': './pseudos',
        'calculation': 'scf',
        'prefix': 'MoS2O'
    },
    'system': {
        'ecutwfc': 60,
        'ecutrho': 500,
        'occupations':'smearing',
        'smearing':'gauss',
        'degauss': 0.01
    },
    'electrons': {
        'conv_thr': 1.0e-8, 
        'mixing_beta': 0.7
    },
} 

calc = Espresso(
    profile=profile,
    pseudopotentials=pseudopotentials,
    tstress=True,  # deprecated, put in input_data
    tprnfor=True,  # deprecated, put in input_data
    input_data=input_data,
    kpts=(1,1,1),
    koffset=(0,0,0))

MoS2OOH.calc = calc

In [186]:
dyn = QuasiNewton(MoS2OOH, trajectory='MoS2OOH.traj')
t = time.time()
dyn.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

                Step[ FC]     Time          Energy          fmax
BFGSLineSearch:    0[  0] 22:11:05   -11238.547013       3.3458
BFGSLineSearch:    1[  2] 22:15:45   -11238.813009       1.2034
BFGSLineSearch:    2[  4] 22:20:35   -11238.829583       0.4732
BFGSLineSearch:    3[  7] 22:26:16   -11239.029844       1.8830
BFGSLineSearch:    4[  8] 22:28:43   -11239.095839       2.5888
BFGSLineSearch:    5[ 10] 22:33:07   -11239.477180       3.4810
BFGSLineSearch:    6[ 12] 22:37:31   -11239.638044       0.9284
BFGSLineSearch:    7[ 13] 22:40:11   -11239.669615       0.4166
BFGSLineSearch:    8[ 14] 22:41:58   -11239.700028       0.5096
BFGSLineSearch:    9[ 15] 22:44:05   -11239.741502       0.9247
BFGSLineSearch:   10[ 16] 22:47:10   -11239.768719       0.6626
BFGSLineSearch:   11[ 17] 22:49:33   -11239.782285       0.4813
BFGSLineSearch:   12[ 19] 22:54:51   -11239.788258       0.5232
BFGSLineSearch:   13[ 20] 22:57:18   -11239.795695       0.3006
BFGSLineSearch:   14[ 21] 22:59:51   -1

In [ ]:
h2_molecule = molecule("H2")
h2_molecule.set_cell(MoS2.get_cell())
h2_molecule.calc = calc
dynH2 = QuasiNewton(h2_molecule, trajectory='H2.traj')
t = time.time()
dynH2.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2 = h2_molecule.get_potential_energy()
print("H2 energy: ", e_h2, " eV")
print("H-H bond lenght: ", h2_molecule.get_distance(0,1))

In [ ]:
h2o_molecule = molecule("H2O")
h2o_molecule.set_cell(MoS2.get_cell())
h2o_molecule.calc = calc
dynH2O = QuasiNewton(h2o_molecule, trajectory='H2O.traj')
t = time.time()
dynH2O.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2o = h2o_molecule.get_potential_energy()
print("H2O energy: ", e_h2o, " eV")
print("O-H bond lenght: ", h2o_molecule.get_distance(0,1))

In [ ]:


"""
Calculations to get the free energy of the O intermediate adsorbed on the Ni surface
"""
o_molecule = Atoms('O', positions=[(0, 0, 0)])
slabNi = fcc111("Ni", a=lattice_constant_Ni, size=(2,2,2))
p = slabNi.positions[4]
o_molecule.translate(p + (0, 0, 1.5))
slabO = slabNi + o_molecule
slabO.center(vacuum=2.0)
constraint = FixAtoms(indices=[0,1,2,3,5,6,7])
slabO.set_constraint(constraint)
view(slabO)

slabO.set_calculator(calculator)
dynO = QuasiNewton(slabO, trajectory="O_Ni.traj", )
t = time.time()
dynO.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

e_slabO = slabO.get_potential_energy()
print("O on Ni(111) energy: ", e_slabO, " eV")
view(slabO)
print("bond Ni-O: ", slabO.get_distance(4,8))


"""
Calculations to get the free energy of the OOH intermediate adsorbed on the Ni surface
"""
ooh_molecule = Atoms('OOH', positions=[(0, 0, 0), (0, 0, 1.4), (0, -0.763, 2.0)])
slabNi = fcc111("Ni", a=lattice_constant_Ni, size=(2,2,2))
p = slabNi.positions[4]
ooh_molecule.translate(p + (0, 0, 1.5))
slabOOH = slabNi + ooh_molecule
slabOOH.center(vacuum=2.0)
constraint = FixAtoms(indices=[0,1,2,3,5,6,7])
slabOOH.set_constraint(constraint)
view(slabOOH)

slabOOH.set_calculator(calculator)
dynOOH = QuasiNewton(slabOOH, trajectory="OOH_Ni.traj", )
t = time.time()
dynOOH.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))

e_slabO = slabOOH.get_potential_energy()
print("OOH on Ni(111) energy: ", e_slabO, " eV")
view(slabOOH)
print("bond Ni-O: ", slabOOH.get_distance(4,8))




"""
Calculations to get the free energy of H2O and H2 needed to calculate the free energies of the reaction
"""

calculator2 = GPAW(xc='RPBE', mode=PW(350), h=0.2, occupations=FermiDirac(0.1))


h2_molecule = molecule("H2")
h2_molecule.set_cell(slab.get_cell())
h2_molecule.center(vacuum=2.0)
h2_molecule.set_calculator(calculator2)
dynH2 = QuasiNewton(h2_molecule, trajectory='H2.traj')
t = time.time()
dynH2.run(fmax=0.05)
print('Calculation time: {} min.'.format((time.time() - t) / 60))
e_h2 = h2_molecule.get_potential_energy()
print("H2 energy: ", e_h2, " eV")
print("H-H bond lenght: ", h2_molecule.get_distance(0,1))